### Проверка работоспособности и качества пайплайна

Здесь мы еще не запускаем никакое API, а загружаем модель (pipeline) напрямую и проверяем на отложенной (тестовой) выборке

In [11]:
import pickle

import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import roc_auc_score

In [2]:
class NumberTaker(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X


class ExperienceTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, column_name):
        self.column_name = column_name
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):  # , y=None
        X = X.copy()
        X.loc[X[self.column_name] == '<1', self.column_name] = 0
        X.loc[X[self.column_name] == '>20', self.column_name] = 21
        X = X.astype(int)
        return X

    
class NumpyToDataFrame(BaseEstimator, TransformerMixin):
    def __init__(self, column_names):
        self.column_names = column_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return pd.DataFrame(X, columns=self.column_names)

In [4]:
with open('models/ctb_clf.pkl', 'rb') as file:
    model = pickle.load(file)

In [7]:
data = pd.read_csv("data/aug_train.csv")
data.head(1)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.92,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0


In [13]:
probs = model.predict_proba(data.drop(columns=['target']))
probs

array([[0.63566818, 0.36433182],
       [0.64433971, 0.35566029],
       [0.35079684, 0.64920316],
       ...,
       [0.65118359, 0.34881641],
       [0.92823604, 0.07176396],
       [0.91253333, 0.08746667]])

In [14]:
preds = model.predict(data.drop(columns=['target']))
preds

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [16]:
roc = roc_auc_score(preds, data['target'])
print(roc)

0.733219019685425
